### 대전 서구, 중구 빵빵지도

In [1]:
system_content = "대전에서 오랜 시간을 보낸 빵 애호가처럼 행동하며, 주어진 정보와 추가적인 검색 결과를 바탕으로 대전의 빵집을 추천합니다. 추천 시에는 주어진 데이터를 참고하고, 각 빵집에서의 유명한 빵 종류를 소개합니다. 중세풍의 로맨스 판타지 소설 속 공주님이 사용할 만한 우아한 말투와 여러 이모티콘을 많이 사용해서 답을 해줘.\n\n# 세부 사항\n- 사용자가 빵집 이름이나 주소 일부를 제공하면 이를 기반으로 추천을 시작합니다.\n- 제공된 정보가 없거나 부족하면 유명한 대전 빵집을 일반적으로 추천합니다.\n- 추천한 빵집마다 다음 정보를 제공하세요:\n    1. 빵집 이름\n    2. 위치 관련 간단한 정보 (주소 일부 또는 주요 지역)\n    3. 유명한 대표 빵의 이름과 간단한 설명\n\n# Steps\n1. 입력된 빵집 이름, 위치 혹은 기타 특징을 확인합니다.\n2. 대전에 실제로 있는 빵집 정보를 확인하거나 검색합니다.\n3. 관련된 빵집 몇 곳을 골라 이름, 위치, 그리고 대표 메뉴를 제공합니다.\n4. 예시의 형태로 사용자에게 결과를 표현합니다.\n\n# Output Format\n- 결과를 항목 리스트 형식으로 작성합니다:\n  - 빵집 이름: [빵집 이름]\n  - 위치: [간단한 주소 또는 지역 설명]\n  - 유명한 빵: \n    1. [빵 이름] - [빵에 대한 간단한 설명]\n    2. [빵 이름] - [빵에 대한 간단한 설명]\n\n# Examples\n\n**Input 1:** 이름에 \"성심당\"이 포함된 빵집을 추천해줘.  \n**Output:**  \n- 빵집 이름: 성심당 본점  \n  - 위치: 대전 중구 은행동 145번지  \n  - 유명한 빵:  \n    1. 튀김소보로 - 겉은 바삭하고 속은 달콤한 소보로 빵  \n    2. 부추빵 - 고소한 빵 속에 담백한 부추와 버터가 어우러진 빵  \n\n- 빵집 이름: 성심당 DCC점  \n  - 위치: 대전 유성구 엑스포로 대전컨벤션센터 내  \n  - 유명한 빵:  \n    1. 크로칸슈 - 달콤한 크림이 가득 든 슈 크로칸  \n    2. 카루소 케이크 - 부드럽고 촉촉한 생크림 케이크  \n\n**Input 2:** 대흥동 쪽에서 유명한 빵집 추천해줘.  \n**Output:**  \n- 빵집 이름: 소담베이커리  \n  - 위치: 대전 중구 대흥동 XX번지  \n  - 유명한 빵:  \n    1. 크림치즈바게트 - 부드럽고 짭짤한 크림치즈가 들어간 바게트  \n    2. 쑥찰떡빵 - 쑥의 향이 풍부한 쫀득한 찰떡이 든 빵 \n\n# Notes\n- 빵집 데이터베이스가 정확하지 않을 경우 추가적으로 검색한다고 가정하고, 유명한 빵집을 중심으로 알려줍니다.\n- 빵집 이름과 대표 빵 설명은 가능한 자세히 작성합니다.\n- 사용자 취향에 따라 주변 유명 빵집 추가도 가능합니다.  \n"


In [ ]:
import requests  # HTTP 요청을 보내기 위한 requests 라이브러리 불러오기
import re
import os
from dotenv import load_dotenv

# .env 파일 로드
load_dotenv()

# 환경 변수 가져오기
endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
api_key = os.getenv("AZURE_OPENAI_API_KEY")
ai_search_endpoint = os.getenv("AI_SEARCH_ENDPOINT")
ai_search_api_key = os.getenv("AI_SEARCH_API_KEY")
ai_search_index_name = os.getenv("AI_SEARCH_INDEX")
ai_search_semantic = os.getenv("AI_SEARCH_SEMANTIC")

def request_gpt(prompt):
    # 요청 헤더 설정
    headers = {
        "Content-Type": "application/json",  # 요청 본문을 JSON 형식으로 지정
        "api-key": api_key  # 인증을 위한 API 키 포함
    }

    # 요청 본문 (채팅 메시지 설정)
    body = {
        "messages": [
            {
                "role": "system",
                "content": system_content
            },
            {
                "role": "user",
                "content": prompt
            }
        ],
        "temperature": 0.9,  # 창의성 조절 (0: 보수적, 1: 창의적)
        "top_p": 0.9,  # 확률 분포 기반 단어 선택 (높을수록 다양한 단어 선택 가능)
        "max_tokens": 4096,
        "stop":None,  
        "stream":False,
        "data_sources": [
            {
            "type": "azure_search",
            "parameters": {
                "endpoint": ai_search_endpoint,
                "index_name": ai_search_index_name,
                "semantic_configuration": ai_search_semantic,
                "query_type": "semantic",
                "fields_mapping": {},
                "in_scope": True,
                "filter": None,
                "strictness": 3,
                "top_n_documents": 5,
                "authentication": {
                "type": "api_key",
                "key": ai_search_api_key
                },
                "key": ai_search_api_key,
            }
            }
        ],
    }


    # OpenAI API에 POST 요청을 보내고 응답을 받아옴
    response = requests.post(endpoint, headers=headers, json=body)
    if response.status_code == 200:
        response_json = response.json()  # 응답을 JSON 형식으로 변환

        # AI의 응답에서 채팅 메시지 추출
        message = response_json['choices'][0]['message']
        citation_list = message['context']['citations']

        # 부적절한 콘텐츠 관리 필터에 걸린 게 있는지 그 결과를 출력하는 것. 수업에서는 활용 안함.
        # content_filter = response_json['choices'][0]['content_filter_results']
        # 역할 (예: assistant)
        role = message['role']
        # 응답 메시지 (AI의 답변)
        content = message['content']
        content = re.sub(r'\[doc(\d+)\]', r'[참조 \1]', content)
        # 역할 출력 (예: assistant)
        print(role)
        return content, citation_list
    else:
        return "", list()
    
# 사용자 입력 프롬프트
# content = request_gpt("안녕 네 이름 뭐야?")
content, citation_list = request_gpt("서구 빵집 알려줘.")

# Jupyter Notebook 환경에서 마크다운 형식으로 AI 응답 표시
from IPython.display import Markdown
display(Markdown(content))

for index in range(len(citation_list)):
    c = citation_list[index]
    print("Index : {}".format(index + 1), c['content'])

assistant


대전 서구에 위치한 빵집들에 대해 알려드리겠습니다. 🍞✨ 이곳들에서 맛있는 빵을 만나보세요!

1. **내가잘가는빵집**
   - **위치**: 대전광역시 서구 신갈마로 153 (갈마동)
   - **대표 빵**: 다양한 종류의 고급스러운 빵이 준비되어 있어요. 특히 바삭하고 촉촉한 크로와상이 인기랍니다! 🥐

2. **만년빵집**
   - **위치**: 대전광역시 서구 만년로 18번길 9, 1층 (만년동)
   - **대표 빵**: 신선한 재료로 만든 크림이 듬뿍 들어간 슈크림빵이 추천 메뉴예요. 🍩

3. **당신을위한빵집**
   - **위치**: 대전광역시 서구 월평북로 11, 1층 (월평동, 주공아파트)
   - **대표 빵**: 갓 구운 부드러운 바게트와 식빵이 특히 유명해요. 🥖

4. **작은빵집 토포(Topo)**
   - **위치**: 대전광역시 서구 원도안로 207, 1층 (도안동)
   - **대표 빵**: 수제 샌드위치와 달콤한 베이글이 인기 만점입니다. 🥯

5. **빵굽는집**
   - **위치**: 대전광역시 서구 문정로 112, 1층 (탄방동)
   - **대표 빵**: 바삭한 페이스트리와 다양한 디저트 빵들이 고객들의 사랑을 받고 있어요. 🥧

각 빵집에서 특별한 맛과 분위기를 즐기실 수 있기를 바랍니다! 💕

Index : 1 대전광역시 서구 신갈마로 153 (갈마동)
내가잘가는빵집
Index : 2 대전광역시 서구 만년로18번길 9, 1층 일부호 (만년동)
만년빵집
Index : 3 대전광역시 서구 월평북로 11, 1층 주공1단지상가 104호 (월평동, 주공아파트)
당신을위한빵집
Index : 4 대전광역시 서구 원도안로 207, 1층 일부호 (도안동)
작은빵집 토포(Topo)
Index : 5 대전광역시 서구 문정로 112 (탄방동, 1층일부)
빵굽는집
